# Homework Lecture 2: Linear Regression

## Preliminaries

### Imports

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.optimize
import sklearn.datasets
import sklearn.linear_model


%matplotlib inline



### Data Directories 

Create a directory with the path below

In [2]:
raw_data_dir="../../raw/california_housing"
data_dir="../../data/probabilisticTools"


### Random Seed

In [3]:
seed=2506
np.random.seed(seed)

### Get Data

<div class="alert alert-block alert-success"> Problem 0 </div>
We download the California housing dataset using the function `sklearn.datasets.fetch_california_housing`.

In [4]:
import sklearn.datasets
housing=sklearn.datasets.fetch_california_housing()

In [5]:
housing.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR'])

In [6]:
print(housing.DESCR)

California housing dataset.

The original database is available from StatLib

    http://lib.stat.cmu.edu/

The data contains 20,640 observations on 9 variables.

This dataset contains the average house value as target variable
and the following input variables (features): average income,
housing average age, average rooms, average bedrooms, population,
average occupation, latitude, and longitude in that order.

References
----------

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
Statistics and Probability Letters, 33 (1997) 291-297.




In [7]:
print(len(housing.feature_names),housing.feature_names)

8 ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [8]:
print(housing.data.shape,housing.target.shape)

(20640, 8) (20640,)


In [9]:
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data["value"]=housing.target
data.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,value
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


## Data Pre-Processing

The variables in `data` have very different scales.
We will replace the values  $x$ on each column by their standarized values 
$$
    z = \frac{x - \bar{x}}{\sigma_x}
$$

<div class="alert alert-block alert-info"> Problem 1.1 </div>
Compute the mean and std deviation of each column in `data`

[HINT] Pandas has convenient functions to compute the column mean an std deviation

In [10]:
data.mean(axis = 0)

MedInc           3.870671
HouseAge        28.639486
AveRooms         5.429000
AveBedrms        1.096675
Population    1425.476744
AveOccup         3.070655
Latitude        35.631861
Longitude     -119.569704
value            2.068558
dtype: float64

In [11]:
data.std(axis = 0)

MedInc           1.899822
HouseAge        12.585558
AveRooms         2.474173
AveBedrms        0.473911
Population    1132.462122
AveOccup        10.386050
Latitude         2.135952
Longitude        2.003532
value            1.153956
dtype: float64

<div class="alert alert-block alert-info"> Problem 1.2 </div>
Create a new `DataFrame` called `data_standarized` the value $x$ of each column gets replaced by its standarized value 
$$
    z = \frac{x - \bar{x}}{\sigma_x}
$$

In [12]:
data_standarized = (data - data.mean(axis = 0))/data.std(axis = 0)

In [13]:
data_standarized[0:10]

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,value
0,2.344709,0.982119,0.628544,-0.153754,-0.974405,-0.049595,1.052523,-1.327803,2.129580
1,2.332181,-0.607004,0.327033,-0.263329,0.861418,-0.092510,1.043159,-1.322812,1.314124
2,1.782656,1.856137,1.155592,-0.049015,-0.820757,-0.025842,1.038478,-1.332794,1.258663
3,0.932945,1.856137,0.156962,-0.049832,-0.766010,-0.050328,1.038478,-1.337785,1.165072
4,-0.012881,1.856137,0.344702,-0.032905,-0.759828,-0.085614,1.038478,-1.337785,1.172871
5,0.087445,1.856137,-0.269723,0.014669,-0.894049,-0.089616,1.038478,-1.337785,0.544598
6,-0.111364,1.856137,-0.200913,-0.306626,-0.292704,-0.090723,1.033796,-1.337785,0.800240
7,-0.395127,1.856137,-0.255226,-0.073540,-0.237073,-0.123473,1.033796,-1.337785,0.299354
8,-0.942336,1.061575,-0.458691,0.044253,-0.193805,-0.100497,1.033796,-1.342777,0.171967
9,-0.094467,1.856137,-0.185279,-0.224682,0.110841,-0.086499,1.033796,-1.337785,0.470071


<div class="alert alert-block alert-info"> Problem 1.3</div>
1. Create a numpy array variable named `X` with all the features (but excluding the house values)
2. Create a numpay array variable named `Y` with the house prices (values)

In [14]:
data.columns.values

array(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population',
       'AveOccup', 'Latitude', 'Longitude', 'value'], dtype=object)

In [20]:
data1 = data_standarized.iloc[:, 0:8]
# Or data1=data_standarized.loc[:, 'MedInc':'Longitude']
X = data1.as_matrix()
X

array([[ 2.34470896,  0.98211887,  0.62854423, ..., -0.04959533,
         1.05252278, -1.32780305],
       [ 2.33218146, -0.60700421,  0.32703343, ..., -0.09250999,
         1.04315928, -1.32281187],
       [ 1.78265622,  1.85613656,  1.15559247, ..., -0.0258419 ,
         1.03847753, -1.33279424],
       ..., 
       [-1.14256563, -0.92482882, -0.09031584, ..., -0.07173277,
         1.77819439, -0.82369324],
       [-1.05455737, -0.84537267, -0.04021014, ..., -0.09122294,
         1.77819439, -0.87360511],
       [-0.78011057, -1.00428498, -0.07044081, ..., -0.0436811 ,
         1.75010387, -0.83367562]])

In [16]:
data2 = data_standarized.iloc[:, -1]
Y = data2.as_matrix()
#type(Y)

## Exact Solution with Numpy

We assume a linear model
$$
     y = \sum_d x_d \theta_d  + \epsilon
$$
where $d$ runs through the housing features and $\epsilon$ is a Gaussian noise term.

<div class="alert alert-block alert-info"> Problem 2.1 </div>
Can you find a reason why we have not included a bias term `b` in the equation?

In [17]:
'''
The reason is that the bias term represents the value of the output y 
when all the values in the feature vector (X) are equal to zero. 

And we have already standardized the data, so y has Gaussian distribution N(0,1)
'''


'\nThe reason is that the bias term represents the value of the output y \nwhen all the values in your feature vector (X) are equal to zero. \n\nAnd we have already standardized the data, so y has Gaussian distribution N(0,1)\n'

<div class="alert alert-block alert-info"> Problem 2.1 </div>
Using only `numpy` matrix algebra functions, find the Maximum Likelihood values of $\theta_d$

[Hint] Computing matrix inverses is computationally expensive.  The function `numpy.lialg.solve` can be used to solve systems of linear equations.

In [21]:
theta_exact=np.linalg.solve(np.dot(X.T,X),np.dot(X.T,Y))
theta_exact

array([ 0.71895227,  0.10291078, -0.23010693,  0.26491789, -0.00390232,
       -0.03408034, -0.77984545, -0.75441522])

<div class="alert alert-block alert-info"> Problem 2.2 </div>
Create a variable named `Y_pred` that for each sample $X$, constains  the maximum likelihood model predicted value for $Y$

In [23]:
Y_pred=np.dot(X,theta_exact)
Y_pred

array([ 1.78784232,  1.65348419,  1.39347822, ..., -1.64417577,
       -1.51604801, -1.34559232])

## Gradient Descent Optimization

We will now solve the same problem using Gradient Descent, instead of the analytic solution.

<div class="alert alert-block alert-info"> Problem 3.1 </div>
Define a python function `mse(theta,X,Y)` that computes the mean square error function given $\theta$, $X$ and $Y$

In [24]:
def mse(theta, X, Y):
    Y_pred = np.dot(X, theta)
    dY = Y_pred - Y
    return 0.5 * np.mean(dY**2)

<div class="alert alert-block alert-info"> Problem 3.2 </div>
Define a python function `grad(theta,X,Y)` that computes the gradient of the error function given $\theta$, $X$ and $Y$

In [25]:
def grad(theta, X, Y):
    Y_pred = np.dot(X, theta)
    dY = (Y_pred - Y)
    return np.dot(X.T, dY) / len(X)

<div class="alert alert-block alert-info"> Problem 3.4 </div>
Using [`numpy.random.normal`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.normal.html) 
generate a random guess of the vector $\theta$ so that each component is $\mathcal{N}(0,1)$ distributed

In [28]:
theta_rand = np.random.normal(size=8)
theta_rand

array([-0.7237708 ,  0.07999463,  0.29075605, -0.04917737,  1.35735786,
       -0.59694298, -0.25401011, -0.28758537])

<div class="alert alert-block alert-info"> Problem 3.3 </div>
Use the function [`scipy.optimze.check_grad`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.check_grad.html)
to verify numerically that `grad` is really the gradient of `mse` for the  $\theta$ guess.

[HINT] `grad` is the gradient of `mse` if `check_grad` returns a very small number (say $\approx 10^-8$)

In [29]:
scipy.optimize.check_grad(mse, grad, theta_rand, X, Y) 
# We can see that the result is a very small number

4.6120718962162211e-08

<div class="alert alert-block alert-info"> Problem 3.4 </div>
** Steepest Descent Algorithm**

1. Pick a value for the learning rate $\eta$
1. Implement the steepest descent update rule
    $$
        \theta \leftarrow \theta - \eta \frac{\partial E}{\partial \theta}
    $$
1. Run the update rule on a loop, starting from your random guess for $\theta$. Repeat  $T=1000$ times
1. Every 100 steps, print the step number and the current error
1. After 1000 steps, print the final error, and the final $\theta$ parameters.
2. If process did not converge, modify value of learning rate $\eta$ and repeat until convergence.

In [30]:
eta = 0.1
T = 1000
theta_SD = theta_rand
for t in range(T):
    if (t % 100 == 0):
        print(t, mse(theta_SD, X, Y))
    theta_SD = theta_SD - eta * grad(theta_SD, X, Y)
print(T, mse(theta_SD, X, Y))
print("theta using Steepest Descent: \n", theta_SD)

0 2.11680623517
100 0.207984361752
200 0.199073952417
300 0.197318181773
400 0.196967096649
500 0.196894874582
600 0.196879231267
700 0.196875545126
800 0.19687456944
900 0.196874275783
1000 0.196874176981
theta using Steepest Descent: 
 [ 0.71852368  0.1028679  -0.22922709  0.26415951 -0.00391236 -0.03406758
 -0.78056052 -0.75507703]


<div class="alert alert-block alert-info"> Problem 3.5 </div>
Compare the MSE of the steepest descent solution to the exact solution.

In [32]:
E_SD = mse(theta_SD, X, Y)
E_exact = mse(theta_exact, X, Y)
print("MSE using Steepest Descent: ", E_SD)
print("Exact MSE: ", E_exact)
print("difference", E_SD - E_exact)

MSE using Steepest Descent:  0.196874176981
Exact MSE:  0.196874118463
difference 5.85177253321e-08


<div class="alert alert-block alert-info"> Problem 3.6 </div>
Compare the  steepest descent parameters $\theta$  to the exact solution.

In [33]:
print("Exact theta \n", theta_exact)
print("Theta using Steepest \n", theta_SD)
print("difference", theta_exact - theta_SD)

Exact theta 
 [ 0.71895227  0.10291078 -0.23010693  0.26491789 -0.00390232 -0.03408034
 -0.77984545 -0.75441522]
Theta using Steepest 
 [ 0.71852368  0.1028679  -0.22922709  0.26415951 -0.00391236 -0.03406758
 -0.78056052 -0.75507703]
difference [  4.28590926e-04   4.28841614e-05  -8.79845111e-04   7.58379225e-04
   1.00373867e-05  -1.27660768e-05   7.15077579e-04   6.61803963e-04]


## Sklearn Comparison

<div class="alert alert-block alert-info"> Problem 4.1 </div>
Use [`sklearn.linear_model.LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
to fit our model.

[Hint] You will need to create a `LinearRegression` object, and the call the `fit` method. Make sure not to fit the intercept (bias).


In [34]:
model = sklearn.linear_model.LinearRegression(fit_intercept=False)
model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

<div class="alert alert-block alert-info"> Problem 4.2 </div>
Compute the mean squared different between the exact model prediction's  `Y_pred`  we saved before and
`sklearn`'s Linear model regression predictions

In [35]:
Y_sk = model.predict(X)
dY=(Y_sk-Y_pred)
np.dot(dY.T,dY)/len(dY)

3.9626724673140836e-30

<div class="alert alert-block alert-info"> Problem 4.3 </div>
Compare the sklearn solution to the exact solution we found earlier.

[Hint] The solution is stored on the model's  `coef_` variable

In [36]:
print('Solution in sklearn', model.coef_)
print('Exact solution', theta_exact)
print('Difference', theta_exact - model.coef_)

Solution in sklearn [ 0.71895227  0.10291078 -0.23010693  0.26491789 -0.00390232 -0.03408034
 -0.77984545 -0.75441522]
Exact solution [ 0.71895227  0.10291078 -0.23010693  0.26491789 -0.00390232 -0.03408034
 -0.77984545 -0.75441522]
Difference [ -7.77156117e-16  -8.32667268e-16   8.04911693e-16  -2.77555756e-16
   2.58473798e-16  -2.35922393e-16  -4.66293670e-15  -5.44009282e-15]


### Statmodels  Comparison

In [37]:
import statsmodels.api as sm

/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


We will solve using  `statmodels` so that we appreciate the difference in emphasis between Machine Learning (`sklearn`) and Statistics Modeling `statmodels` 

<div class="alert alert-block alert-info"> Problem 5.1 </div>
Use [`statmodels.api.OLS`](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html) to solve the same linear regression problem


In [38]:
model = sm.OLS(Y, X)
results = model.fit()

<div class="alert alert-block alert-info"> Problem 5.2 </div>
Compare the `statmodels` solution to the exact solution we found earlier.

[Hint] The fitted parameters are stored on the results 's  `parms` variable

In [39]:
print("feature, theta,theta_exact")
for idx in range(X.shape[1]):
    print(idx, results.params[idx], theta_exact[idx])

feature, theta,theta_exact
0 0.718952272225 0.718952272225
1 0.102910779714 0.102910779714
2 -0.23010693263 -0.23010693263
3 0.264917894141 0.264917894141
4 -0.00390232364271 -0.0039023236427
5 -0.0340803412556 -0.0340803412556
6 -0.779845445551 -0.779845445551
7 -0.754415222097 -0.754415222097


<div class="alert alert-block alert-info"> Problem 5.3 </div>
Print a  `statmodels` result summary (function `summary` of the results object).

It will show you a number of estimates on goodness-of-fit, significance of coefficients, etc.

In [40]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     3971.
Date:                Sun, 04 Feb 2018   Prob (F-statistic):               0.00
Time:                        17:17:46   Log-Likelihood:                -19668.
No. Observations:               20640   AIC:                         3.935e+04
Df Residuals:                   20632   BIC:                         3.942e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.7190      0.007    104.056      0.0

### Independent test for categorial variables

<div class="alert alert-block alert-info"> Problem 6.1 </div>

Read the data from file 'homework.csv' in the  'data_dir' directory

Perform a $\chi^2$ test of independence between the variables `X` and `Y`.
Are 'X' and 'Y' dependent on each other?

[Hint] You can copy any code you need from the [`CategoricalInference`](./CategoricalInference.ipynb) Notebook,
but make sure to import any python modules you may need.

In [41]:
import pandas as pd
import numpy as np
import scipy.special as special

In [42]:
dat = pd.read_csv(data_dir + '/' + 'homework.csv')
X=dat["X"]
Y=dat["Y"]
X.shape,Y.shape

((1000,), (1000,))

In [43]:
Z_x=pd.get_dummies(X)
print("Z_x.shape =",Z_x.shape)
Z_x=Z_x.as_matrix()

Z_x.shape = (1000, 4)


In [44]:
Z_y=pd.get_dummies(Y)
print("Z_x.shape =",Z_y.shape)
Z_y=Z_y.as_matrix()
Z_y[:5]

Z_x.shape = (1000, 5)


array([[0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [45]:
def C2_independence(Z_x,Z_y):
    N=len(Z_x)
    D=Z_x.shape[1]
    K=Z_y.shape[1]
    # p_y has index k
    p_y=Z_y.mean(axis=0)
    # p_x has index d
    p_x=Z_x.mean(axis=0)
    # p will be K*D, with indexes k,d
    p=p_y[:,np.newaxis]*p_x[np.newaxis,:]
    # Z_y has indexes i,k and Z_x has indexes i,d
    #Z will be N*K*D, with indexes i,k,d
    Z=Z_y[:,:,np.newaxis]*Z_x[:,np.newaxis,:]
    # sum over i, left with a K*D matrix
    obs=Z.sum(axis=0) # last two expressions are the same as np.dot(Z_y^T,Z_x)
    # expect
    expect=N*p
    df=obs-expect
    df2=df*df
    c2 = (df2/np.maximum(1e-9,expect)).sum()
    return c2,special.chdtrc((K-1)*(D-1),c2)

In [46]:
C2_independence(Z_x,Z_y)

(10.345900312733516, 0.58564326482281204)

In [ ]:
# The result of χ2 is 0.585 > 0.5, so we cannot regect null hypothesis, which is they are independent. 